In [10]:
import numpy as np
import matplotlib.pyplot as plt
import torch  as th
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pickle
import pandas as pd
import seaborn as sns
import random
import os
import glob
import numpy as np
import gzip as gz
import pickle
from tqdm import tqdm
from transformers import AutoModel, AutoConfig
from torch.utils.data import TensorDataset, DataLoader, random_split
from BESTRq_classes.BESTRq import BestRqFramework, RandomProjectionQuantizer
from compute_fft import compute_spectrogram, plot_spectrogram, mask_and_replace
from models.CNN_BiLSTM_Attention import ParallelModel

# GAN implementation : comparison between spectrum and temporal serie 
https://towardsdatascience.com/synthetic-time-series-data-a-gan-approach-869a984f2239

In [11]:
def make_net(model, n_layers, hidden_units, output_units):
    for i in range(n_layers):
        model.add_module(f'GRU_{i + 1}', nn.GRU(hidden_units, hidden_units, batch_first=True))
        model.add_module('OUT', nn.Sequential(nn.Linear(hidden_units, output_units), nn.Sigmoid()))
    return model

In [12]:
class Supervisor(nn.Module):
    def __init__(self, hidden_dim):
        super(Supervisor, self).__init__()
        self.hidden_dim = hidden_dim
        self.model = self.build()

    def build(self):
        model = nn.Sequential()
        return make_net(model,
                        n_layers=2,
                        hidden_units=self.hidden_dim,
                        output_units=self.hidden_dim)


In [13]:
class Generator(nn.Module):
    def __init__(self, hidden_dim, net_type='GRU'):
        super(Generator, self).__init__()
        self.hidden_dim = hidden_dim
        self.net_type = net_type
        self.model = self.build()

    def build(self):
        layers = []
        if self.net_type == 'GRU':
            layers.append(nn.GRU(input_size=self.hidden_dim, hidden_size=self.hidden_dim, num_layers=3, batch_first=True))
        else:
            # Autre type de réseau de neurones récurrent
            pass
        return nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

class Discriminator(nn.Module):
    def __init__(self, hidden_dim, net_type='GRU'):
        super(Discriminator, self).__init__()
        self.hidden_dim = hidden_dim
        self.net_type = net_type
        self.model = self.build()

    def build(self):
        layers = []
        if self.net_type == 'GRU':
            layers.append(nn.GRU(input_size=self.hidden_dim, hidden_size=self.hidden_dim, num_layers=3, batch_first=True))
        else:
            # Autre type de réseau de neurones récurrent
            pass
        layers.append(nn.Linear(self.hidden_dim, 1))
        return nn.Sequential(*layers)

    def forward(self, x):
        return th.sigmoid(self.model(x))

class Recovery(nn.Module):
    def __init__(self, hidden_dim, n_seq):
        super(Recovery, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_seq = n_seq
        self.model = self.build()

    def build(self):
        layers = []
        layers.append(nn.Linear(self.hidden_dim, self.hidden_dim))
        layers.append(nn.ReLU())
        layers.append(nn.Linear(self.hidden_dim, self.n_seq))
        return nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

class Embedder(nn.Module):
    def __init__(self, hidden_dim):
        super(Embedder, self).__init__()
        self.hidden_dim = hidden_dim
        self.model = self.build()

    def build(self):
        layers = []
        layers.append(nn.Linear(self.hidden_dim, self.hidden_dim))
        layers.append(nn.ReLU())
        layers.append(nn.Linear(self.hidden_dim, self.hidden_dim))
        return nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

class Supervisor(nn.Module):
    def __init__(self, hidden_dim):
        super(Supervisor, self).__init__()
        self.hidden_dim = hidden_dim
        self.model = self.build()

    def build(self):
        layers = []
        layers.append(nn.Linear(self.hidden_dim, self.hidden_dim))
        layers.append(nn.ReLU())
        layers.append(nn.Linear(self.hidden_dim, self.hidden_dim))
        return nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

In [20]:
# Définition des dimensions
latent_dim = 128  # Dimension de l'espace latent
input_dim = 28*28  # Dimension des données d'entrée, par exemple pour des images MNIST

# Initialisation du GAN
generator = Generator(hidden_dim=128)
discriminator = Discriminator(hidden_dim=128)
supervisor = Supervisor(hidden_dim=128)

# Entraînement du GAN (à implémenter)
# Vous devrez définir la fonction d'entraînement du GAN en alternant entre
# l'entraînement du générateur, du discriminateur et du superviseur.

# Génération de données
def generate_data(n_samples):
    # Génération de vecteurs aléatoires dans l'espace latent
    noise = th.randn(n_samples, latent_dim)
    # Utilisation du générateur pour générer de nouvelles données
    generated_data = generator(noise)
    return generated_data

# Génération de 10 exemples de données
generated_samples = generate_data(10)
print(generated_samples[0].shape)

torch.Size([10, 128])
